不正アカウントのラベル付け

In [1]:
import numpy as nm
import pandas as pd
import os
import random
from datetime import datetime
import networkx as nx
from ripser import ripser
from persim import PersistenceImager
import matplotlib.pyplot as plt
import time

In [2]:
# sample.csvのあるディレクトリを指定
os.chdir("/Users/sinco/Desktop/python")

# CSVファイルの読み取り
df = pd.read_csv('2018-transaction-3.txt')  # トランザクションデータ
df_phish = pd.read_csv('phish.csv')  # フィッシングアカウント一覧

# フィッシングアカウントのリストを取得
phish_list = df_phish["account"].values.tolist()
df['is_phish'] = df['from_address'].isin(phish_list) | df['to_address'].isin(phish_list)
df_ph = df[df['is_phish']]  # フィッシングアカウントに関連するトランザクションを抽出

# 新しいフォルダを作成 (日時で一意に)
folder_name = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_path = f"/Users/sinco/Desktop/python/pimgs/{folder_name}/"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# ノーマルアカウントのデータを抽出
df_normal = df[~df['is_phish']]  # フィッシングアカウントでないトランザクションを抽出

# 抽出する行数と割合を指定
num_samples = 1000  # 抽出する総行数
phish_ratio = 0.3   # フィッシングアカウントの割合 (0.3 = 30%)

# フィッシングとノーマルの行数を計算
num_phish_samples = int(num_samples * phish_ratio)
num_normal_samples = num_samples - num_phish_samples

# フィッシングアカウントとノーマルアカウントをランダムにサンプリング
df_ph_sample = df_ph.sample(n=min(num_phish_samples, len(df_ph)), random_state=42)  # フィッシングアカウント
df_normal_sample = df_normal.sample(n=min(num_normal_samples, len(df_normal)), random_state=42)  # ノーマルアカウント

# サンプルを結合
df_sample = pd.concat([df_ph_sample, df_normal_sample])

In [3]:
# グラフを作成
edges = pd.DataFrame({
    'source': df_sample['from_address'],
    'target': df_sample['to_address'],
    'weight': df_sample['value']
})
G = nx.from_pandas_edgelist(edges, edge_attr=True)

# ノード位置を計算
pos = nx.spring_layout(G)
A = nm.array(list(pos.values()))  # ノードの座標を配列に変換


In [4]:
# TDAに必要な図を生成
#diagrams = ripser(A)['dgms']

# パーシステンスイメージの設定
pimgr = PersistenceImager(pixel_size=0.05)
pimgr.birth_range = (0, 1)
pimgr.pers_range = (0, 1)
pimgr.fit(diagrams[1], skew=True)

# ランダムに選ぶ点の数と試行回数を設定
num_samples = 10  # 各イテレーションで選ぶ点の総数
num_iterations = 5  # イテレーションの数
phish_ratio = 0.2  # フィッシングアカウントの割合

NameError: name 'diagrams' is not defined

In [ ]:
# 各試行でノードをマスキング
for i in range(1, num_iterations + 1):
    print(f"Iteration {i}")

    start = time.perf_counter()

    # フィッシングアカウントとノーマルアカウントのサンプル数を計算
    num_phish_samples = int(num_samples * phish_ratio)
    num_normal_samples = num_samples - num_phish_samples

    if num_phish_samples > len(df_ph):
        num_phish_samples = len(df_ph)  # 利用可能なフィッシングアカウント数を超えないように調整
    if num_normal_samples > (len(df) - len(df_ph)):
        num_normal_samples = len(df) - len(df_ph)  # 利用可能なノーマルアカウント数を超えないように調整

    # フィッシングアカウントとノーマルアカウントをサンプリング
    sampled_phish = df_ph.sample(n=num_phish_samples)
    sampled_normal = df.drop(df_ph.index).sample(n=num_normal_samples)  # ノーマルアカウントをサンプリング

    # マスキングするノードを取得
    masked_nodes = pd.concat([sampled_phish['from_address'], sampled_phish['to_address'], 
                              sampled_normal['from_address'], sampled_normal['to_address']]).unique()

    # グラフをマスキング
    masked_edges = edges[edges['source'].isin(masked_nodes) & edges['target'].isin(masked_nodes)]
    G_masked = nx.from_pandas_edgelist(masked_edges, edge_attr=True)

    # マスキングしたグラフのノード位置を再計算
    pos_masked = nx.spring_layout(G_masked)
    A_masked = nm.array(list(pos_masked.values()))  # ノードの座標を配列に変換

    # TDAを適用
    diagrams_masked = ripser(A_masked)['dgms']

    # 画像の生成
    pimgs = pimgr.transform(diagrams_masked, skew=True)
    pimgr.weight_params = {'low': 0.0, 'high': 1.0, 'start': 2.0, 'end': 10.0}

    # 画像の保存
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    name = f"case_{i}.jpg"
    print(f"Saving image as: {name}")
    pimgr.plot_image(pimgs[1], ax, out_file=os.path.join(folder_path, name))

    elapsed_time = time.perf_counter() - start
    print(f"Iteration {i} completed in {elapsed_time:.4f} seconds")
    plt.close(fig)  # プロットを閉じる

print("All images saved successfully.")